In [0]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv",header=None)
df = df.drop(df.columns[0], axis=1)
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,7.33,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,2.82,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [0]:
arr = df.values
print(arr.shape)
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

(24983, 100)
1810455 (1810455,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr[idx[0][0], idx[1][0]]

-0.34

In [0]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 3

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
latent_item_features

array([[0.20284828, 0.88660967, 0.15142429],
       [0.01084659, 0.08558042, 0.93482848],
       [0.73331089, 0.5429106 , 0.41777738],
       [0.72864947, 0.82034104, 0.11099062],
       [0.07317649, 0.01325505, 0.44701351],
       [0.17104913, 0.36156228, 0.35057307],
       [0.13754116, 0.02705355, 0.60444123],
       [0.1478479 , 0.34688232, 0.9450098 ],
       [0.49645608, 0.22672036, 0.69426521],
       [0.85345723, 0.33704774, 0.4804791 ],
       [0.06683314, 0.89519856, 0.93358421],
       [0.11891   , 0.65081049, 0.97940542],
       [0.49662061, 0.82322826, 0.91091802],
       [0.64137043, 0.96057167, 0.63080891],
       [0.31578196, 0.97431939, 0.84732493],
       [0.45893919, 0.78629774, 0.96373064],
       [0.79824033, 0.76354021, 0.94350432],
       [0.25892121, 0.01198383, 0.2657825 ],
       [0.40984859, 0.19918787, 0.18739229],
       [0.24059759, 0.37471097, 0.40266026],
       [0.99432445, 0.78877267, 0.60904157],
       [0.82010181, 0.7989824 , 0.13168284],
       [0.

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=70):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)


In [0]:
sgd()

25.853751888339442
23.54171670180513
21.895679036275297
19.074868144871342
17.446813530797634
17.195183777740674
17.065578672189478


In [0]:
n_latent_factors = 3

user_ratings = idx.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 9.76565766e+01,  6.46084229e+00,  3.52548744e+00, ...,
         3.60924859e-01,  2.23667032e+00,  1.55709495e+00],
       [ 4.90062309e+01,  6.66554828e+00,  7.15602320e+00, ...,
         7.52185144e+00,  7.19232259e+00,  8.29674253e+00],
       [ 4.74724643e+01, -6.40066392e-02,  1.25817037e+00, ...,
         3.72924048e+00,  1.51435712e+00,  3.70907082e+00],
       ...,
       [ 3.89010509e+01, -7.85480245e-01, -1.10665268e+00, ...,
        -7.52777481e-01, -1.36955672e+00, -7.12687370e-01],
       [ 3.81791737e+01,  6.15950099e-02, -4.57247206e-01, ...,
        -5.01705527e-01, -7.75175452e-01, -3.54832364e-01],
       [ 7.19760329e+01,  2.05670648e+00,  2.68871704e+00, ...,
         4.34518687e+00,  2.64150305e+00,  4.67358157e+00]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [0]:
comparison_data

,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,"(100.0, 97.65657661039273)","(4.08, 6.460842288911663)","(-0.29, 3.525487436971272)","(6.36, 5.238592038059707)","(4.37, 2.4471288085429603)","(-2.38, 0.6544232254381401)","(-9.66, 2.8706153120864766)","(-0.73, 1.1967039845962417)","(-5.34, 1.8438287311522334)","(8.88, 6.005418382799629)","(9.22, 7.284583502557828)","(6.75, 4.052583716803293)","(8.64, 2.1106781251949425)","(4.42, 0.26377247143331495)","(7.43, 1.5679515080898452)","(4.56, 1.53205647473751)","(-0.97, 0.7857553663500272)","(4.66, -1.7072949863138387)","(-0.68, 2.4174310207662337)","(3.3, 1.0902822807880703)","(-1.21, -0.6847231886135765)","(0.87, 1.7820169370172072)","(8.64, 4.490197926404411)","(8.35, 5.290289910891263)","(9.17, 4.531128415736386)","(0.05, 3.4099454692478064)","(7.57, 1.33861987709565)","(4.71, 2.6945608417324762)","(0.87, 2.901272409509801)","(-0.39, 2.2075502294036577)","(6.99, 3.5192801655060935)","(6.5, 1.9516148910600948)","(-0.92, 2.813430457003982)","(7.14, 3.0472869284029405)","(9.03, 2.2900396926725213)","(-1.8, 2.7007983762785943)","(0.73, 3.3963222844272303)","(7.09, 4.519358840800994)","(3.4, 5.866685464403377)","(-0.87, 1.54470599546192)",...,"(-6.7, 2.3546406383391765)","(-3.35, 3.6814175349557146)","(-9.03, 3.376425641416054)","(4.47, 5.41971510816576)","(4.08, 1.6833653732915015)","(-3.83, 2.924507674297121)","(8.74, 5.242510955213005)","(1.12, 2.865820672074087)","(0.78, 2.476450358207174)","(7.52, 3.363621974467371)","(-5.0, 0.2172596849912543)","(2.77, 2.3163227706203253)","(8.3, 0.5865239865060875)","(7.77, 0.5054094259957047)","(7.33, 0.6095020779492752)","(6.21, 2.6727097915183107)","(7.72, 1.1701911022111313)","(8.98, 1.7315343247503756)","(8.64, 0.018751819774789843)","(8.2, 0.14668117986036025)","(3.93, 2.8524539796261212)","(4.85, 2.967231433715202)","(4.85, 1.358546416489338)","(6.07, 2.5761117252108514)","(8.98, 1.666242914595638)","(4.51, 5.345924703586511)","(-0.05, 3.215980294876312)","(3.69, 4.580915958342103)","(4.56, 4.043301426664118)","(0.58, 2.4897150613263284)","(2.82, 4.686492759184068)","(-4.95, 2.679044814027589)","(-0.29, 3.4955979687830507)","(7.86, 5.786471247834599)","(-0.19, 4.387578025465847)","(-2.14, 4.143109841002394)","(3.06, 4.207345799251399)","(0.34, 0.36092485891864473)","(-4.32, 2.236670323433884)","(1.07, 1.5570949457427166)"
1,"(49.0, 49.006230864031544)","(99.0, 6.665548277591709)","(99.0, 7.156023199221247)","(99.0, 7.304327994121216)","(99.0, 7.25774919026938)","(9.03, 5.258760748146393)","(9.27, 8.585373586694788)","(9.03, 4.525813098460666)","(9.27, 2.6296717635080054)","(99.0, 7.406372372491472)","(99.0, 7.015291958468568)","(7.33, 8.268026086540246)","(7.57, 8.886174617684793)","(9.37, 2.481498242312082)","(6.17, 8.851897021048897)","(-6.36, 1.7340393433176164)","(-6.89, 1.2778967568512114)","(-7.86, 3.0156053907976736)","(9.03, 3.797410698429501)","(9.03, 5.462930094720652)","(9.03, 5.139196188847552)","(7.28, 8.508206410594084)","(99.0, 8.385467145905045)","(8.25, 7.542098175228487)","(99.0, 6.464537284016617)","(99.0, 8.090210202081897)","(7.48, 8.554215744106278)","(7.28, 7.659626584737903)","(7.28, 8.135989749493943)","(8.93, 7.234535752912721)","(99.0, 8.208988150154456)","(6.17, 8.136868515855738)","(7.28, 7.658334800678208)","(99.0, 7.140736714088837)","(99.0, 8.132593670294309)","(8.98, 7.918872693162141)","(7.33, 7.267325449880401)","(99.0, 5.891308634060182)","(6.17, 7.928593157463129)","(9.08, 8.501999207177262)",...,"(6.46, 8.539601445766108)","(7.28, 8.218378032650113)","(99.0, 7.743040315939654)"